In [ ]:
from kafka import KafkaConsumer
from kafka import KafkaProducer
import sys
import json
import time
import joblib
import pandas as pd
%run CategorizeBloodPressure.ipynb  # Execute the notebook

In [ ]:

# Setting up the Python consumer
bootstrap_servers = ['localhost:9092']
input_topic = 'Topic_A'
output_topic = 'Topic_C'  # Topic for ML model output
#group_id = 'my_group_id1'
consumer = KafkaConsumer(input_topic, bootstrap_servers=bootstrap_servers,
                         auto_offset_reset='earliest', value_deserializer=lambda x: json.loads(x.decode('utf-8')))

# Setting up the Kafka producer for ML model output
#ml_producer = KafkaProducer(bootstrap_servers=bootstrap_servers)
ml_producer = KafkaProducer(bootstrap_servers=bootstrap_servers, value_serializer=lambda v: json.dumps(v).encode('utf-8'))


In [ ]:
# Load the regression model
regression_model = joblib.load('regression_model.joblib')

In [ ]:
# Reading messages from the input topic and producing results to the output topic
try:
    for message in consumer:
        input_data = message.value
        print(f"Received input data: {input_data}")


        # Extract features based on your input data
        selected_features_sbp = [input_data.get("PP", ""), input_data.get("CO", "")]
        selected_features_dbp = [input_data.get("PP", ""), input_data.get("CO", "")]

        # Create a DataFrame with the selected features
        features_sbp = pd.DataFrame([selected_features_sbp], columns=['PP', 'CO'])
        features_dbp = pd.DataFrame([selected_features_dbp], columns=['PP', 'CO'])

        # Make predictions using the regression model
        prediction_sbp_dbp = regression_model.predict(pd.concat([features_sbp, features_dbp], axis=1))

        # Create the result dictionary
        prediction_result = {
            "Predicted_SBP": prediction_sbp_dbp[0, 0],
            "Predicted_DBP": prediction_sbp_dbp[0, 1],
            "BloodPressureCategory": categorize_blood_pressure(prediction_sbp_dbp[0, 0], prediction_sbp_dbp[0, 1])
        }

        # Concatenate input_data and prediction_result
        output_data = {**input_data, **prediction_result}

        print(f"Prediction result and and blood pressure category: {output_data}")

        # Produce the result to the output topic
        ml_producer.send(output_topic, value=output_data)

except KeyboardInterrupt:
    pass

finally:
    #Close the consumer and producer
    consumer.close()
    ml_producer.close()